### Подключение библиотек

In [5]:
import os
import pandas as pd
import numpy as np #версия 1.16

import nltk
from nltk.stem.snowball import *
stemmer = SnowballStemmer('russian')

from gensim.models import word2vec, KeyedVectors
from gensim.test.utils import get_tmpfile, datapath

from tempfile import TemporaryFile

import matplotlib.pyplot as plt
from matplotlib import style
style.use("ggplot")

from sklearn.cluster import KMeans

import pickle

In [7]:
cwd = os.getcwd()
cwd
path = '/'
os.chdir(path)

### Загрузка исходных сообщений

In [ ]:
%%time
df = pd.read_csv('name.csv', encoding='cp1251')
df.shape

### Предобработка исходных сообщений, сохранение их в .csv

In [11]:
%%time 
def review_to_words(raw_review):
    #without_comments = re.sub("слово", "", raw_review)
    without_comments = raw_review.split('Сообщение:') 
    if len(without_comments) > 1:
        without_comments.pop()
        without_comments.append('Сообщение')
    without_comments = " ".join(without_comments)
    letters_only = re.wub("[^a-zA-Zа-яА-Я]", " ", without_comments)
    words = letters_only.lower().split()
    meaningful_words = [w for w in words if len(w) > 1]
    singles = [stemmer.stem(word) for word in meaningful_words]
    return (" ".join(singles))

CPU times: user 8 µs, sys: 2 µs, total: 10 µs
Wall time: 13.6 µs


In [ ]:
%%time
processed = [review_to_words(text) for text in df["SendMsg"]]

In [ ]:
%%time 
df_new = pd.DataFrame(processed)

In [ ]:
%%time
df_new.to_csv('name_new.csv', encoding='cp1251')

### Создание word2vec-векторов 

In [12]:
def build_corpus(data):
    corpus = []
    for sentence in data:
        word_list = sentence.split(" ")
        corpus.append(word_list)
    return corpus

In [ ]:
%%time
processed = pd.read_csv('name_new.csv', encoding='cp1251')

In [ ]:
%%time
corpus = build_corpus(processed['0'])

In [ ]:
%%time
model = word2vec.Word2Vec(corpus, size=30, window=3, min_count=1, workers=4)

In [ ]:
fname = get_tmpfile('name.kv')

In [ ]:
%%time
#запись в файл
model.wv.save(fname)

In [ ]:
#чтение из файла
word_vectors2 = KeyedVectors.load(fname, mmap='r')

In [ ]:
#создание словаря со словами и соответствующими векторами 
w2v = dict(zip(word_vectors2.wv.index2word, word_vectors2.wv.syn0))

In [ ]:
#кол-во разных слов в модели 
len(word_vectors2.wv.vocab)

In [ ]:
#функция для создания векторов предложений
def vector_for_sentence(w2v_dict, data):
    return np.array([
        np.mean([w2v_dict[w] for w in sentence if w in w2v_dict]
        or [np.zeros(50)], axis=0)
         for sentence in data
    ])

In [ ]:
%%time data_vectors = vector_for_sentence(w2v, corpus)

In [ ]:
data_vectors.shape

In [ ]:
#имя файла для сохранения numpy array
filename = get_tmpfile('new_file')

### Создание модели k-means

In [ ]:
fname = path + ''

In [ ]:
%%time
X = np.load(fname)

kmeans_all = KMeans(n_clusters=50)
kmeans_all.fit()

pkl_filename = path + ''
with open(pkl_filename, 'wb') as file:
    pickle.dump(kmeans_all, file)

### Просмотр кластеров

In [ ]:
df = pd.read_csv(fname, sep='', encoding='')

pkl_filename = pat + ''
with open(pkl_filename, 'rb') as file:
    kmeans_all = pickle.load(file)

In [ ]:
labels_all = kmeans_all.labels_
dct_all = {}
for i in laels_all:
    if i in dct_all:
        dct_all[i] += 1
    else:
        dct_all[i] = 1
dct_all

In [ ]:
out = pd.DataFrame(columns=['MSG'])
for i in range(len(labels_all)):
    if labels_all[i] == 0:
        out.loc[i, 'MSG'] = df['SendMsg']

In [ ]:
for i in out['MSG'][:10000]:
    print(i)